In [17]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pandas as pd
import gensim
from gensim import corpora
from nltk.corpus import stopwords
import spacy


data = pd.read_csv("data.csv")
print(data['content'].info())
dataFiltered = data["content"][:35000]

stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
dataFiltered=dataFiltered.apply(remove_stopwords)

# nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

text_list=dataFiltered.tolist()
#print(text_list[5])
tokenized_reviews = lemmatization(text_list)
#print(tokenized_reviews[5])

dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, id2word=dictionary, num_topics=15, random_state=100, chunksize=100, passes=10)
print("-"*10, "\n", "Topics\n", ldamodel.print_topics(num_words=10))



<class 'pandas.core.series.Series'>
RangeIndex: 50000 entries, 0 to 49999
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
50000 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB
None


c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use

---------- 
 Topics
 [(0, '0.048*"russian" + 0.039*"intelligence" + 0.031*"news" + 0.028*"medium" + 0.020*"story" + 0.019*"information" + 0.018*"reporter" + 0.015*"source" + 0.014*"tweet" + 0.014*"press"'), (1, '0.055*"police" + 0.035*"officer" + 0.033*"missile" + 0.027*"death" + 0.022*"korean" + 0.016*"man" + 0.016*"victim" + 0.015*"incident" + 0.014*"crime" + 0.014*"murder"'), (2, '0.090*"%" + 0.034*"percent" + 0.032*"year" + 0.026*"number" + 0.025*"study" + 0.021*"." + 0.018*"high" + 0.017*"state" + 0.015*"rate" + 0.012*"population"'), (3, '0.025*"company" + 0.020*"money" + 0.017*"tax" + 0.017*"job" + 0.015*"year" + 0.015*"business" + 0.014*"budget" + 0.013*"program" + 0.012*"government" + 0.010*"worker"'), (4, '0.026*"investigation" + 0.022*"official" + 0.018*"court" + 0.017*"law" + 0.016*"case" + 0.015*"federal" + 0.013*"administration" + 0.013*"attorney" + 0.013*"statement" + 0.012*"order"'), (5, '0.016*"city" + 0.016*"day" + 0.012*"water" + 0.011*"flight" + 0.011*"m." + 0.011*"h

In [18]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis


C:\Users\eslam\AppData\Roaming\Python\Python310\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.096485  0.054385       1        1  17.336742
14     0.096102 -0.051728       2        1  15.026771
6      0.078454 -0.179420       3        1   9.460715
4     -0.118806 -0.145538       4        1   7.988432
3      0.152962 -0.138797       5        1   7.576957
5      0.018114  0.142393       6        1   7.113690
12    -0.151743 -0.045159       7        1   6.100590
10    -0.016914  0.257097       8        1   4.723738
0     -0.170936  0.015254       9        1   4.482243
1     -0.229553  0.118160      10        1   4.326178
2      0.175902 -0.009086      11        1   3.580776
8     -0.079875 -0.161976      12        1   3.473723
9      0.003250 -0.073297      13        1   3.306476
13    -0.070387  0.055493      14        1   3.237527
7      0.216945  0.162219      15        1   2.265440, topic_info=           Term          Freq         Total Category  logprob  loglift
11024         %  19367.000000  19367.000000  Default  30.0000  30.0000
696       woman  20415.000000  20415.000000  Default  29.0000  29.0000
273     country  35082.000000  35082.000000  Default  28.0000  28.0000
219    campaign  25563.000000  25563.000000  Default  27.0000  27.0000
503      police  14191.000000  14191.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
647       third    961.357685   3405.434703  Topic15  -4.9504   2.5226
439       major   1095.040193   7464.669855  Topic15  -4.8202   1.8680
1512      point   1113.471451  10487.677361  Topic15  -4.8035   1.5447
2285        top   1047.594284   9526.569945  Topic15  -4.8645   1.5798
535      record    969.335283   6413.669091  Topic15  -4.9421   1.8978

[985 rows x 6 columns], token_table=       Topic      Freq Term
term                       
11024     11  0.999957    %
1879       5  0.244375    .
1879       6  0.036223    .
1879       9  0.012206    .
1879      11  0.585739    .
...      ...       ...  ...
704       10  0.000268   ’s
704       11  0.004218   ’s
704       12  0.013056   ’s
704       13  0.019216   ’s
704       15  0.009240   ’s

[2457 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 15, 7, 5, 4, 6, 13, 11, 1, 2, 3, 9, 10, 14, 8])

In [19]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.856757852698726

Coherence Score:  0.5152407686124983
